In [1]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"muhammadabdullah0078","key":"1150ec8fd2e044b014b59811a4120797"}'}

In [5]:
!mkdir -p ~/.kaggle

In [6]:
!mv kaggle.json ~/.kaggle/

mv: cannot stat 'kaggle.json': No such file or directory


In [7]:
!chmod 600 ~/.kaggle/kaggle.json

In [8]:
!kaggle datasets list -s leapgestrecog


ref                                        title                                    size  lastUpdated                 downloadCount  voteCount  usabilityRating  
-----------------------------------------  ---------------------------------  ----------  --------------------------  -------------  ---------  ---------------  
gti-upm/leapgestrecog                      Hand Gesture Recognition Database  2286085038  2018-07-30 06:43:29.133000          38048        666  0.75             
maychondouglas/hands-gestures-augmentated  Hands Gestures Augmentated         6411036418  2020-06-18 04:22:24.643000             15          1  0.125            
maychondouglas/novoleapgestrecog           NOVOleapGestRecog                  2494219076  2020-06-19 00:00:00.670000             13          0  0.1875           


In [9]:
!kaggle datasets download -d gti-upm/leapgestrecog


Dataset URL: https://www.kaggle.com/datasets/gti-upm/leapgestrecog
License(s): CC-BY-NC-SA-4.0
100% 2.12G/2.13G [00:26<00:00, 243MB/s]
100% 2.13G/2.13G [00:26<00:00, 86.3MB/s]


In [10]:
!unzip -q leapgestrecog.zip -d dataset/


In [11]:
!ls dataset/leapGestRecog


00  01	02  03	04  05	06  07	08  09


In [15]:
import os, cv2, random, pandas as pd

src_root = "dataset/leapGestRecog/"
dst_root = "clean_dataset/"
os.makedirs(dst_root, exist_ok=True)

# your final 4-gesture mapping
gesture_map = {
    "01_palm": "open",   # Light ON
    "03_fist": "fist",   # Light OFF
    "05_thumb": "up",    # Brightness increase
    "10_down": "down"    # Brightness decrease
}

IMG_SIZE = (64, 64)
data_rows = []

for subject in sorted(os.listdir(src_root)):        # 00..09
    subj_path = os.path.join(src_root, subject)
    if not os.path.isdir(subj_path): continue
    for gesture in sorted(os.listdir(subj_path)):   # gesture folders
        if gesture not in gesture_map:
            continue
        label = gesture_map[gesture]
        out_dir = os.path.join(dst_root, label)
        os.makedirs(out_dir, exist_ok=True)

        files = os.listdir(os.path.join(subj_path, gesture))
        random.shuffle(files)
        take = files[:300]   # limit to ~300 per gesture

        for f in take:
            in_path = os.path.join(subj_path, gesture, f)
            img = cv2.imread(in_path)
            if img is None:
                continue
            img = cv2.resize(img, IMG_SIZE)
            out_name = f"{subject}_{gesture}_{f}"
            out_path = os.path.join(out_dir, out_name)
            cv2.imwrite(out_path, img)
            data_rows.append({"image_path": out_path, "label": label})

# save the labels to csv
df = pd.DataFrame(data_rows)
df.to_csv("gesture_labels.csv", index=False)
print("Cleaning done! Total images:", len(df))
df.head()


Cleaning done! Total images: 8000


,image_path,label
0,clean_dataset/open/00_01_palm_frame_00_01_0124...,open
1,clean_dataset/open/00_01_palm_frame_00_01_0142...,open
2,clean_dataset/open/00_01_palm_frame_00_01_0159...,open
3,clean_dataset/open/00_01_palm_frame_00_01_0139...,open
4,clean_dataset/open/00_01_palm_frame_00_01_0030...,open


In [16]:
!zip -r clean_dataset.zip clean_dataset/


Streaming output truncated to the last 5000 lines.
  adding: clean_dataset/fist/06_03_fist_frame_06_03_0075.png (deflated 11%)
  adding: clean_dataset/fist/04_03_fist_frame_04_03_0125.png (deflated 12%)
  adding: clean_dataset/fist/05_03_fist_frame_05_03_0121.png (deflated 13%)
  adding: clean_dataset/fist/08_03_fist_frame_08_03_0001.png (deflated 11%)
  adding: clean_dataset/fist/07_03_fist_frame_07_03_0062.png (deflated 12%)
  adding: clean_dataset/fist/08_03_fist_frame_08_03_0165.png (deflated 13%)
  adding: clean_dataset/fist/01_03_fist_frame_01_03_0016.png (deflated 11%)
  adding: clean_dataset/fist/08_03_fist_frame_08_03_0099.png (deflated 12%)
  adding: clean_dataset/fist/06_03_fist_frame_06_03_0107.png (deflated 11%)
  adding: clean_dataset/fist/07_03_fist_frame_07_03_0024.png (deflated 12%)
  adding: clean_dataset/fist/00_03_fist_frame_00_03_0148.png (deflated 11%)
  adding: clean_dataset/fist/03_03_fist_frame_03_03_0039.png (deflated 11%)
  adding: clean_dataset/fist/00_03_fi

In [18]:
from google.colab import files
files.download("clean_dataset.zip")
files.download("gesture_labels.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>